Deep Learning Assignment 3 

References:
Code source 1: https://medium.com/swlh/stock-price-prediction-with-pytorch-37f52ae84632


In [1]:
import pandas as pd

data = pd.read_csv("AAPL_2006-01-01_to_2018-01-01.csv")
data

,Date,Open,High,Low,Close,Volume,Name
0,2006-01-03,10.34,10.68,10.32,10.68,201853036,AAPL
1,2006-01-04,10.73,10.85,10.64,10.71,155225609,AAPL
2,2006-01-05,10.69,10.70,10.54,10.63,112396081,AAPL
3,2006-01-06,10.75,10.96,10.65,10.90,176139334,AAPL
4,2006-01-09,10.96,11.03,10.82,10.86,168861224,AAPL
...,...,...,...,...,...,...,...
3014,2017-12-22,174.68,175.42,174.50,175.01,16349444,AAPL
3015,2017-12-26,170.80,171.47,169.68,170.57,33185536,AAPL
3016,2017-12-27,170.10,170.78,169.71,170.60,21498213,AAPL
3017,2017-12-28,171.00,171.85,170.48,171.08,16480187,AAPL


In [2]:
from sklearn.preprocessing import MinMaxScaler
price = data[['Close']]
scaler = MinMaxScaler(feature_range=(-1, 1))
price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [3]:
price

,Close
0,-0.959333
1,-0.958979
2,-0.959924
3,-0.956732
4,-0.957205
...,...
3014,0.983331
3015,0.930843
3016,0.931198
3017,0.936872


In [5]:
import numpy as np
def split_data(stock, lookback):
    data_raw = stock.to_numpy() # convert to numpy array
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]

lookback = 20 # choose sequence length
x_train, y_train, x_test, y_test = split_data(price, lookback)

In [6]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2399, 19, 1)
(2399, 1)
(600, 19, 1)
(600, 1)


In [7]:
import torch
import torch.nn as nn
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)

In [14]:
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 100

In [15]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [16]:
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [17]:
import time
hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []
for t in range(num_epochs):
    y_train_pred = model(x_train)
    loss = criterion(y_train_pred, y_train_lstm)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
training_time = time.time()-start_time
print("Training time: {}".format(training_time))

Epoch  0 MSE:  0.5906404852867126
Epoch  1 MSE:  0.43337318301200867
Epoch  2 MSE:  0.2729237675666809
Epoch  3 MSE:  0.12233077734708786
Epoch  4 MSE:  0.3617275357246399
Epoch  5 MSE:  0.1729903221130371
Epoch  6 MSE:  0.1104547530412674
Epoch  7 MSE:  0.13606098294258118
Epoch  8 MSE:  0.16169512271881104
Epoch  9 MSE:  0.16986508667469025
Epoch  10 MSE:  0.16291993856430054
Epoch  11 MSE:  0.14542743563652039
Epoch  12 MSE:  0.12166845798492432
Epoch  13 MSE:  0.0959242433309555
Epoch  14 MSE:  0.07281968742609024
Epoch  15 MSE:  0.056432243436574936
Epoch  16 MSE:  0.04769522324204445
Epoch  17 MSE:  0.04215472191572189
Epoch  18 MSE:  0.032755788415670395
Epoch  19 MSE:  0.023896588012576103
Epoch  20 MSE:  0.02778245136141777
Epoch  21 MSE:  0.028741175308823586
Epoch  22 MSE:  0.021386312320828438
Epoch  23 MSE:  0.012874709442257881
Epoch  24 MSE:  0.008568307384848595
Epoch  25 MSE:  0.007945106364786625
Epoch  26 MSE:  0.008936001919209957
Epoch  27 MSE:  0.01033806335180997

In [18]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn) = self.gru(x, (h0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [19]:
model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [20]:
import time
hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []
for t in range(num_epochs):
    y_train_pred = model(x_train)
    loss = criterion(y_train_pred, y_train_lstm)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
training_time = time.time()-start_time
print("Training time: {}".format(training_time))

Epoch  0 MSE:  0.44344866275787354
Epoch  1 MSE:  0.19219444692134857
Epoch  2 MSE:  0.1272525191307068
Epoch  3 MSE:  0.16925549507141113
Epoch  4 MSE:  0.1050383672118187
Epoch  5 MSE:  0.06657137721776962
Epoch  6 MSE:  0.06142337620258331
Epoch  7 MSE:  0.0565544031560421
Epoch  8 MSE:  0.035820722579956055
Epoch  9 MSE:  0.00840685237199068
Epoch  10 MSE:  0.005562870763242245
Epoch  11 MSE:  0.02881527878344059
Epoch  12 MSE:  0.019189655780792236
Epoch  13 MSE:  0.007053830660879612
Epoch  14 MSE:  0.012720762751996517
Epoch  15 MSE:  0.01835642382502556
Epoch  16 MSE:  0.01439759973436594
Epoch  17 MSE:  0.0057145701721310616
Epoch  18 MSE:  0.0006934403791092336
Epoch  19 MSE:  0.003055660054087639
Epoch  20 MSE:  0.008107227273285389
Epoch  21 MSE:  0.008931153453886509
Epoch  22 MSE:  0.005735555663704872
Epoch  23 MSE:  0.00337175908498466
Epoch  24 MSE:  0.0039126938208937645
Epoch  25 MSE:  0.00555171025916934
Epoch  26 MSE:  0.00582509208470583
Epoch  27 MSE:  0.00405733